<h1><p align = 'center'> Use o ambiente "raspilum" (Python 3.10.18)</h4></p>


<h5> <P align='justify'>Nas células a seguir, faço todas as importações e processos iniciais necessários, como calibração e proteção de ferramentas. </p></h5>

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [15]:
from jubilee_controller import JubileeMotionController
import matplotlib.pyplot as plt
import time
import numpy as np 
from pynput import keyboard
import cv2
from datetime import datetime
from camera_controller import camera_tool
from email_sender import send_email
from image_processing import detect_circle
from gripper_controller import Gripper
from micropipette_controller import Micropipette

In [3]:
jubilee = JubileeMotionController()

In [6]:
jubilee.reset()

Reconectando...


In [5]:
jubilee.home_all(mesh_mode_z=False)
jubilee.protect_tools(on=True)
jubilee.move_xyz_absolute(z=50)

<h2><strong>Câmera</strong>

<h5> <P align='justify'> As células a seguir fazem a defição o objeto da câmera e sua instalação</p></h5>

In [7]:
camera = camera_tool(jubilee)

In [10]:
camera.install()

<h5> <P align='justify'> Usando o método "photo" salva a imagem. É necesário ajustar o foco antes, então recomendo que ative o ambiente <code>raspilum</code> no terminal e rode o código <code>camera.py</code>. Isso deve abrir o vídeo da câmera de forma contínua em outra janela. Depois de ajustar, pode fechar o vídeo apertando a tecla "q". Para tirar fotos com a câmera, use o comando "photo", como exemplificado a baixo.</p></h5>


In [12]:
jubilee.keyboard_controll() #Abra a câmera pelo terminal antes.

X:   60.00 | Y:  190.00 | Z:  220.00 || Passo: 10  mm
Interrompido


In [13]:
camera.photo("Foto qualquer.png")

<h5> <P align='justify'>Após o uso, guarde a ferramenta usando o método <code>uninstall</code>.</p></h5>


In [14]:
camera.uninstall()

<h2><strong>Micropipeta</strong>

In [17]:
pipeta = Micropipette(jubilee,parking_position_xy=(138,18),move_velocity=3000)

<h5> <P align='justify'>Quando o método <code>install</code> é utilizado, a altura mínima no eixo z é redefinida para 160, permitindo que haja espaço suficiente para a movimentação da pipetadora com a ponteira. Caso a altura seja menor que 160, a plataforma é descida automaticamente.</p></h5>

In [21]:
pipeta.install()

<h5> <p align='justify'>Coloque com béquer de 50ml de capacidade com 40 ml de água no centro da plataforma. <br>Adicione a ponteira na pipetadora manualmente. Esse é um processo que pode ser feito de forma automática, posicionando a caixa de ponteira e conferindo as coordenadas corretas para o encaixe, mas para esse exemplo, faça isso manualmente.</p></h5>

In [33]:
jubilee.move_xyz_absolute(z=230)
jubilee.move_xyz_absolute(x=165,y=250,velocity=3000)

<h5> <p align='justify'>Confira se a ponteira está alinhada com o béquer. Caso esteja, pode rodar a próxima célula, se não, então faça o alinhamento. <br> A próxima célula usa o método <code>press</code> da micropipeta, o qual recebe um valor de volume a ser sugado em microlitros. Em seguida, o béquer é levantado, o método <code>aspirate</code> da micropipeta é utilizado, fazendo com que o líquido seja sugado, e então o béquer é abaixado. </p></h5>

In [ ]:
pipeta.press(200)
jubilee.move_xyz_absolute(z=165)
pipeta.aspirate()
jubilee.move_xyz_absolute(z=220)

<h5> <p align='justify'>Por fim, uso o método <code>dispense</code> para liberar o líquido. É importante ressaltar que a depender a viscosidade do líquido, ele pode começar a formar uma bolha na ponta caso fique muito tempo parado sem ser dispensado. </p></h5>

In [35]:
pipeta.dispense()


<h5> <p align='justify'>Vamos agora fazer um loop de coleta e dispensa de líquido </p></h5>

In [37]:
for i in range(100,1300,300):
    print(f"{i} ul")
    pipeta.press(i)
    jubilee.move_xyz_absolute(z=165)
    pipeta.aspirate()
    jubilee.move_xyz_absolute(z=220)
    pipeta.dispense()


100 ul
400 ul
700 ul
1000 ul


<h5> <p align='justify'>Para liberar a ponteira, use o método <code>eject_tip</code></p></h5>

In [38]:
pipeta.eject_tip()

<h5> <p align='justify'>E após o uso, guarde a ferramenta usando o seu método de desinstalação</p></h5>

In [41]:
pipeta.uninstall()